In [1]:
import numpy as np
import pandas as pd
import math
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
class Dataset():
    def __init__(self):
        pass
    
    def load_data(self):
        path = r"../input/datasets-for-regression/Bank_Customer_Churn_Prediction/Bank_Customer_Churn_Prediction.csv"
    
        df = pd.read_csv(path, header=None, delimiter=",")
        df = df.drop(0, axis = 1)
        features = df.iloc[1:, 0:10]
        labels = df.iloc[1:, -1]
        return (features, labels)

    def one_hot_encode(self, features):
        encoded_country = pd.get_dummies(features[2])
        encoded_gender = pd.get_dummies(features[3])
    
        merged_columns = pd.concat([encoded_country, encoded_gender], axis = "columns")
        features = pd.concat([features, merged_columns], axis = "columns")
        features = features.drop([2, 3], axis = "columns")
        return features
    
    def split_data(self, features, labels):
        total_samples = features.shape[0]
        feature_columns = features.columns.values.tolist()
        test_split_size = int(np.ceil((20 / 100) * total_samples))
    
        train_x, test_x, train_y, test_y = train_test_split(features, labels, test_size = test_split_size)

        train_x = train_x.reset_index(drop = True)
        test_x = test_x.reset_index(drop = True)
        train_y = train_y.reset_index(drop = True)
        test_y = test_y.reset_index(drop = True)
    
        train_y = train_y.astype(float)
        test_y = test_y.astype(float)

        for column in feature_columns:
            train_x[column] = train_x[column].astype(float)
            test_x[column] = test_x[column].astype(float)

        return (train_x, test_x, train_y, test_y)
    
    def min_max_normalization(self, df):
        normalized_df = (df - df.min()) / (df.max() - df.min())
        return normalized_df

In [3]:
class Neural_Network():
    def __init__(self, epochs, lr, hidden_nodes):
        self.epochs = epochs
        self.lr = lr
        self.hidden_nodes = hidden_nodes
        
        # Getting features and labels
        dataset = Dataset()
        self.features, self.labels = dataset.load_data()
        print(f"Fetures Before One Hot Encoding \n{self.features}\n" + "-" * 150, "\n")
        print(f"Labels \n{self.labels}\n" + "-" * 150, "\n")
        
        # Applying One Hot Encoding
        self.features = dataset.one_hot_encode(self.features)
        print(f"Fetures After One Hot Encoding \n{self.features}\n" + "-" * 150, "\n")
        
        # Spilitting data into training and testing subsets
        self.train_f, self.test_f, self.train_l, self.test_l = dataset.split_data(self.features, self.labels)
        print(f"Training Features \n{self.train_f}\n" + "-" * 150, "\n")
        print(f"Testing Features \n{self.test_f}\n" + "-" * 150, "\n")
        print(f"Training Labels \n{self.train_l}\n" + "-" * 150, "\n")
        print(f"Testing Labels \n{self.test_l}\n" + "-" * 150, "\n")
        
        # Normalizing features
        self.train_f = dataset.min_max_normalization(self.train_f)
        self.test_f = dataset.min_max_normalization(self.test_f)
        print(f"Training Features After Min-Max Normalization \n{self.train_f}\n" + "-" * 150, "\n")
        print(f"Testing Features After Min-Max Normalization \n{self.test_f}\n" + "-" * 150, "\n")
        
        # Initializing weights and biases for both layers
        self.weights_layer1, self.weights_layer2, self.biases_layer1, self.biases_layer2 = self.init_parameters(13, self.hidden_nodes, 1)
        
    
    def forward_pass(self, inputs, weights_layer1, weights_layer2, biases_layer1, biases_layer2):
        # Weighted sum for layer 1
        weighted_sums_layer1 = self.calculate_weighted_sum(weights_layer1, input_features)
        weighted_sums_layer1 = self.add_biases(weighted_sums_layer1, biases_layer1)
        
        # Applying Relu
        for index in range(self.hidden_nodes):
            weighted_sum_layer1[index] = self.relu(weighted_sum_layer1[index])
        
        # Weighted sum for layer 2
        weighted_sums_layer2 = self.calculate_weighted_sum(weights_layer2, weighted_sums_layer1)
        weighted_sums_layer2 = self.add_biases(weighted_sums_layer2, biases_layer2)
        
        # Applying Sigmoid
        y_hat = self.sigmoid(weighted_sums_layer2)
        return y_hat, weighted_sums_layer1
    
    def calculate_weighted_sum(weights, inputs):
        weighted_sum = np.dot(weights, inputs)
        return weighted_sum

    def add_biases(weighted_sum, biases):
        return np.add(weighted_sum, biases)
    
    def backward_pass(self, y_hat, weighted_sums_layer1):
        # Calculate gradient of output node
        error_O = y_hat * (1 - y_hat) * (self.train_l - y_hat)
        
        # Calculate gradient of hidden nodes
        t1 = np.product(self.weights_layer2, error_O)
        t2 = np.product(weighted_sums_layer1, (1 - weighted_sums_layer1))
        errors_H = t1 * t2
        
        # Calculating new parameters for layer 2
        weights_layer2 = np.sum(self.weights_layer2, np.product(self.lr, np.product(error_O, weighted_sums_layer1)))
        biases_layer2 = np.sum(self.biases_layer2, np.product(self.lr, error_O))
        
        # Calculating new parameters for layer 1
        weights_layer1 = np.sum(self.weights_layer1, np.product(self.lr, np.product(errors_H, self.train_f)))
        biases_layer1 = np.sum(self.biases_layer1, np.product(self.lr, errors_H))
        
        return (weights_layer1, weights_layer2, biases_layer1, biases_layer2)
    
    
    def fit(self):
        for epoch in range(self.epochs):
            y_hat, weighted_sums_layer1 = self.forward_pass(self.train_f, self.weights_layer1, self.weights_layer2, self.biases_layer1, self.biases_layer2)
            
            total_train_samples = self.train_f.shape[0]
            training_cost = self.cost_function(self.train_l, y_hat, total_train_samples)
            print(f"Training Cost at Epoch [{epoch + 1}] is {training_cost}")
            
            self.weights_layer1, self.weights_layer2, self.biases_layer1, self.biases_layer2 = self.backward_pass(y_hat, weighted_sums_layer1)
    
    def init_parameters(self, input_features, hidden_nodes, output_nodes):
        num_weights_l1 = input_features * hidden_nodes
        num_weights_l2 = hidden_nodes * output_nodes
        num_biases_l1 = hidden_nodes
        num_biases_l2 = output_nodes
        
        weights_layer1 = []
        weights_layer2 = []
        biases_layer1 = []
        biases_layer2 = []
        
#         temp.append(round(random.uniform(-0.5, 0.5), 1))
        
        return (weights_layer1, weights_layer2, biases_layer1, biases_layer2)
    
    
    def cost_function(self, y, y_hat, total_samples):
        cost = (-1 / total_samples) * np.sum(y * np.log(0.0001 + y_hat) + (1 - y) * np.log(0.0001 + 1 - y_hat))
        return cost
    
    def sigmoid(self, z):
        z = 1 / (1 + np.exp(-z))
        return z

    def relu(self, z):
        return np.maximum(z, 0)

In [4]:
# epochs = int(input("Epochs: "))
# hidden_nodes = int(input("Hidden Units: "))
# lr = float(input("Learning Rate: "))
network = Neural_Network(10, 0.0001, 3)
# network.fit()

Fetures Before One Hot Encoding 
        1        2       3   4   5          6  7  8  9          10
1      619   France  Female  42   2          0  1  1  1  101348.88
2      608    Spain  Female  41   1   83807.86  1  0  1  112542.58
3      502   France  Female  42   8   159660.8  3  1  0  113931.57
4      699   France  Female  39   1          0  2  0  0   93826.63
5      850    Spain  Female  43   2  125510.82  1  1  1    79084.1
...    ...      ...     ...  ..  ..        ... .. .. ..        ...
9996   771   France    Male  39   5          0  2  1  0   96270.64
9997   516   France    Male  35  10   57369.61  1  1  1  101699.77
9998   709   France  Female  36   7          0  1  0  1   42085.58
9999   772  Germany    Male  42   3   75075.31  2  1  0   92888.52
10000  792   France  Female  28   4  130142.79  1  1  0   38190.78

[10000 rows x 10 columns]
----------------------------------------------------------------------------------------------------------------------------------------

In [5]:
# network_weights, network_biases = init_parameters(13, num_hidden_units, 1)
# print(f"Network Weights for Both Layers \n{network_weights}\n" + "-" * 150, "\n" + f"Network Biases for Both Layers \n{network_biases}")